In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def cleaner(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.findAll('p')

    text = ''
    p = re.compile('<.*?>')
    for paragraph in paragraphs:
        text += re.sub(p, '', str(paragraph))
    return text

In [3]:
def create_csv(label, data):
    df = pd.DataFrame(data=data, columns=['title', 'url', 'text'])
    df.to_csv(path_or_buf='./data/' + label + '.csv', index=False)

In [ ]:
banks_en = ['absolutbank', 'akbars', 'alfabank', 'avangard',
            'bcs-bank', 'binbankcreditcard', 'binbank', 'bnpparibaseast',
            'bspb', 'citibank', 'crediteuropebank', 'deltacredit',
            'fk_otkritie', 'gazprombank', 'homecreditbank', 'jugra',
            'locko-bank', 'minbank', 'mkb', 'modulbank',
            'mosoblbank', 'mts-bank', 'otpbank', 'pochtabank',
            'promsvyazbank', 'qiwibank', 'raiffeisen', 'rencredit',
            'rgsbank', 'rosbank', 'roscap', 'rsb',
            'rshb', 'rusfinancebank', 'sberbank', 'skb-bank',
            'smpbank', 'sovcombank', 'tcs', 'touchbank',
            'transcapitalbank', 'trust', 'ubrr', 'unicreditbank',
            'uralsib', 'v-express-bank', 'vozrozhdenie', 'vtb24',
            'vtb', 'yandexdengi']
banks_ru = ['абсолют+банк', 'ак+барс+банк', 'альфа+банк', 'авангард+банк',
            'бкс+банк', 'бинбанк+кредит+кард', 'бинбанк', 'бнп+париба+восток',
            'банк+санкт+петербург', 'ситибанк', 'кредит+европа+банк', 'дельта+кредит',
            'фк+открытие', 'газпромбанк', 'хоум+кредит+банк', 'югра',
            'локо+банк', 'московский+индустриальный+банк', 'московский+кредитный+банк', 'модульбанк',
            'мособлбанк', 'мтс+банк', 'отп+банк', 'почта+банк',
            'промсвязьбанк', 'киви+банк', 'райффайзен', 'ренессанс+кредит',
            'росгосстрах+банк', 'росбанк', 'российский+капитал', 'русский+стандарт+банк',
            'россельхозбанк', 'русфинанс+банк', 'сбербанк', 'скб+банк',
            'смп+банк', 'совкомбанк', 'тинькофф', 'touch+bank',
            'транскапиталбанк', 'траст+банк', 'уральский+банк+реконструкции+и+развития', 'юникредит+банк',
            'уралсиб', 'восточный+экспресс+банк', 'возрождение+банк', 'втб24',
            'втб', 'яндекс+деньги']
unused_banks_ru = ['бинбанк+кредит+кард', 'российский+капитал']

In [ ]:
url = None
urls = set()
response = None
prefix = 'https://lenta.ru/search/v2/process?from='
suffix = '0&size=10&sort=2&title_only=0&domain=1&modified,format=yyyy-MM-dd&bloc=4&query='

max_news = 20
for i in range(len(banks_en)):
    if (banks_ru[i] not in unused_banks_ru):
        bank_news = []
        continue_parsing = True
        index = 0
        while continue_parsing:
            if index == 0:
                url = prefix + suffix + banks_ru[i]
            else:
                url = prefix + str(index) + suffix + banks_ru[i]
            index += 1

            response = requests.get(url)
            if response.ok and len(response.json()['matches']) != 0:
                for news in response.json()['matches']:
                    if news['url'] not in urls:
                        urls.add(news['url'])
                        bank_news.append({'title': news['title'], 'url': news['url'], 'text': cleaner(news['url'])})
                        if len(bank_news) == max_news:
                            continue_parsing = False
                            break
            else:
                continue_parsing = False

        create_csv(banks_en[i], bank_news)

In [4]:
url = None
urls = set()
response = None
prefix = 'https://lenta.ru/search/v2/process?from='
suffix = '0&size=100&sort=2&title_only=0&domain=1&modified,format=yyyy-MM-dd&bloc=4&query=банк'

index = 0
err = 0
new_data = True
bank_news = []
while(new_data and len(urls) < 100000):
    if index % 10 == 0:
        print('index: {}   size: {}'.format(index, len(urls)))
    
    if index == 0:
        url = prefix + suffix
    else:
        url = prefix + str(index) + suffix
    index += 1
    new_data = False
    
    response = requests.get(url)
    if response.ok and len(response.json()['matches']) != 0:
        for news in response.json()['matches']:
                    if news['url'] not in urls:
                        new_data = True
                        urls.add(news['url'])
                        bank_news.append({'title': news['title'], 'url': news['url'], 'text': cleaner(news['url'])})
    
    if not new_data and err < 3:
        err += 1
        new_data = True

index: 0   size: 0
index: 10   size: 190
index: 20   size: 290
index: 30   size: 390
index: 40   size: 490
index: 50   size: 590
index: 60   size: 690
index: 70   size: 790
index: 80   size: 890
index: 90   size: 990
index: 100   size: 1090
index: 110   size: 1190
index: 120   size: 1290
index: 130   size: 1390
index: 140   size: 1490
index: 150   size: 1590
index: 160   size: 1690
index: 170   size: 1790
index: 180   size: 1890
index: 190   size: 1990
index: 200   size: 2090
index: 210   size: 2190
index: 220   size: 2290
index: 230   size: 2390
index: 240   size: 2490
index: 250   size: 2590
index: 260   size: 2690
index: 270   size: 2790
index: 280   size: 2890
index: 290   size: 2990
index: 300   size: 3090
index: 310   size: 3190
index: 320   size: 3290
index: 330   size: 3390
index: 340   size: 3490
index: 350   size: 3590
index: 360   size: 3690
index: 370   size: 3790
index: 380   size: 3890
index: 390   size: 3990
index: 400   size: 4090
index: 410   size: 4190
index: 420   si

In [5]:
create_csv('news', bank_news)